In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut

showInfo = 0
p = ut.leer_csv("data/ARIA/4.PatientDemographics/_Patient.csv", showInfo).drop(columns = ["SSN","WorkPhone","HomePhone",
                      "Citizenship","MobilePhone","CreationDate","Language","PatientId2","MobilePhoneProviderSer","PatientId",
                       "FileDataSiteID","PatientUID"])

pa = ut.leer_csv("data/ARIA/4.PatientDemographics/_PatientAddress.csv", showInfo).drop(columns = ["PrimaryFlag"])

a  = ut.leer_csv("data/ARIA/4.PatientDemographics/_Address.csv", showInfo).drop(columns = ["County","AddressLine1",
                     "AddressLine2","PhoneNumber1","PhoneNumber2","EMailAddress","OriginationDate",
                      "TerminationDate","PostalCode","AddressType","StateOrProvince"])

tr = pd.read_csv("data/ARIA_merged&selected/TreatmentRecord.csv", index_col="TreatmentRecordSer")\
.drop(columns = ["ActualMachineAuthorization","MachOverrideFlag"])

# pp = ut.leer_csv("data/ARIA/4.PatientDemographics/_PatientParticular.csv", showInfo).drop(columns = ["MaritalStatus",
#          "RetireDate","DeathDate","PassportNumber","UserDefAttrib01","UniversalPatientId","RetireReason","RetireNote"])

In [2]:
# ut.info_tabla(a)

In [3]:
join1 = pa.merge(a, on="AddressSer", how="left", suffixes = ("_pa", "_a")).groupby("PatientSer", as_index=False).first()
join2 = p.merge(join1, on="PatientSer", how='left', suffixes = ("_p", "_paa")).drop(columns="AddressSer")
# join3 = join2.merge(pp, on="PatientSer", how="left")
join2.head()

,FirstName,DateOfBirth,PatientSer,Sex,Country,CityOrTownship
0,Truebeam,NaN,4,Other,None,None
1,NaN,4/29/2008 12:00:00 AM,7,Unknown,None,None
2,NaN,4/29/2008 12:00:00 AM,8,Unknown,None,None
3,NaN,5/19/2008 12:00:00 AM,9,Unknown,None,None
4,NaN,4/29/2008 12:00:00 AM,11,Unknown,None,None


In [4]:
join2["DateOfBirth"] = pd.DatetimeIndex( pd.to_datetime(join2["DateOfBirth"])).year
join2.head()

,FirstName,DateOfBirth,PatientSer,Sex,Country,CityOrTownship
0,Truebeam,NaN,4,Other,None,None
1,NaN,2008.0,7,Unknown,None,None
2,NaN,2008.0,8,Unknown,None,None
3,NaN,2008.0,9,Unknown,None,None
4,NaN,2008.0,11,Unknown,None,None


# Separar pacientes reales y "de prueba"

In [5]:
l2 = list(pd.read_csv("data/nombres_hombre.csv", skiprows=6, delimiter=";").Nombre.values)
l3 = list(pd.read_csv("data/nombres_mujer.csv" , skiprows=6, delimiter=";").Nombre.values)

import unidecode
l4 = []
for i in l2+l3:
    if isinstance(i, str): l4.append(unidecode.unidecode(i))

In [6]:
nombres = join2.FirstName.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') \
            .str.upper().str.split().str[0]

In [7]:
patient_real = join2[ nombres.isin(l4)]
patient_test = join2[~nombres.isin(l4)]
print(len(patient_real))
print(len(patient_test))

6759
165


# Corrección
Reviso aquellos pacientes en 'patient_test' que han sido tratados en las máquinas, por si hubiera quedado alguno como falso negativo.

In [8]:
idx = list(set.intersection(set(tr.PatientSer.unique()), set(patient_test.PatientSer)  ))
aa = patient_test[patient_test.PatientSer.isin(idx)]
aa.shape

(51, 6)

Parece que dentro de este grupo aquellos con 'CityOrTownship' no nulo son reales.

In [9]:
idx_pats_real = list(set.union(set(patient_real.PatientSer), set(aa[aa.CityOrTownship.notnull()].PatientSer)))

In [10]:
patient_real = join2[ join2.PatientSer.isin(idx_pats_real)]
patient_test = join2[~join2.PatientSer.isin(idx_pats_real)]

In [11]:
# patient_real
print(join2.shape)

print(patient_real.shape)
print(patient_test.shape)
# patient_real.head()

(6924, 6)
(6789, 6)
(135, 6)


# Guardar tablas

In [12]:
patient_real.to_csv("data/ARIA_merged&selected/Patient_real.csv", index=False)
patient_test.to_csv("data/ARIA_merged&selected/Patient_test.csv", index=False)